In [1]:
import numpy as np
import pandas as pd
import spacy
import nltk 
import requests
import json
import googletrans
import random
import re

from random import randint
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from spacy.lang.en import English
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import wordnet 
from googletrans import Translator

spacy.load('en_core_web_sm')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

analyser = SentimentIntensityAnalyzer()
parser = English()
porter = PorterStemmer()
translator = Translator()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ninagroot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
file = 'edit-db_model1.xlsx'
df_gratitude = pd.read_excel(file, 'gratitude')
df_exercises = pd.read_excel('exercises_model1.xlsx')
df_journal = pd.read_excel('prepared_journal_model1_lang.xlsx')
#df_journal = df_journal.dropna(subset = ['id'])

topics = {
    'social media': ['social', 'media', 'network', 'twitter', 'facebook', 'instagram', 'web', 'linkedin', 'google', 'viral', 'google+', 'event', 'website', 'reach', 'telegraph', 'telephone', 'blog', 'internet', 'social', 'sociality', 'technoself', 'intranet', 'cyber', 'business', 'sociable', 'activist', 'microblogging', 'socially', 'cybernetic', 'pinterest', 'youtube', 'tumblr', 'cyberspace', 'extranet', 'telnet', 'cybernetics', 'modem', 'communication', 'informatics', 'virtual', 'party', 'picture'],
    'body insecurity': ['body', 'insecure', 'belly', 'skin', 'waist', 'hair', 'eyes', 'lips', 'ears', 'forehead', 'eyebrow', 'chin', 'scar', 'nose', 'look', 'eat', 'ate', 'boldness', 'beard', 'smile', 'teeth', 'stomach', 'back', 'ass', 'butt', 'frame', 'feature', 'physique', 'bod', 'bodies', 'torso', 'arm', 'head', 'corpse', 'chest', 'shape', 'shoulder', 'form', 'neck', 'structure', 'leg', 'mass', 'weight', 'shaky', 'unattractive', 'unsure', 'anxious', 'jealous', 'unconfident', 'distrustful', 'uncomfortable', 'unsecured', 'neurotic', 'hopeless', 'depressed', 'introverted', 'helpless', 'fragile', 'danger', 'anxiety', 'mistrust', 'instability', 'unrest', 'hardship', 'vulnerability', 'lawlessness', 'uncertainty', 'turmoil', 'strife', 'fragility', 'hopelessness', 'malnutrition', 'alienation', 'violence', 'poverty', 'tension', 'shortages', 'despair', 'chaos', 'distrust', 'resentment', 'underdevelopment', 'anarchy', 'paranoia', 'cynicism', 'fear', 'scared', 'insecure', 'fearful'],
    'social anxiety': ['fear', 'worry', 'stress', 'nervousness', 'attention', 'shame', 'panic', 'social', 'attack', 'own', 'activities', 'lonely', 'alone', 'rejection', 'distance', 'careful', 'courage', 'conversation',  'afraid', 'mistake', 'stupid', 'insecurity', 'public', 'speaking', 'nausea', 'stutter', 'acting', 'performance', 'stage', 'fright', 'fear', 'public', 'shyness', 'blushing', 'anxiety', 'self-consciousness', 'panic', 'illness', 'mood disorder', 'timidness', 'social functioning', 'parties', 'outgoing', 'street', 'public', 'shyness', 'blushing', 'anxiety', 'self-consciousness', 'panic', 'illness', 'mood disorder', 'timidness', 'street', 'outside', 'social functioning', 'parties', 'outgoing', 'illness', 'peer rejection', 'stutter', 'stage fright', 'guilt', 'panic attack'],
    'friendship' : ['best', 'friend', 'classmate', 'schoolmate', 'roomie', 'boyfriend', 'sister', 'amigo', 'acquanitance', 'brother', 'comerade', 'girlfriend', 'person', 'pal', 'schoolfriend', 'mate', 'colleague', 'roommate', 'love', 'pal', 'buddy', 'talk', 'soulmate', 'fight', 'trust', 'understanding', 'happiness', 'relations', 'empathy', 'company', 'feelings', 'unity', 'friendship', 'friend', 'companionship', 'relationship', 'affection', 'break', 'broke']
    }

NameError: name 'pd' is not defined

In [2]:
display(df_journal)

NameError: name 'df_journal' is not defined

In [3]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def lemma1(word):
    return WordNetLemmatizer().lemmatize(word)

def lemma2(word):
    return WordNetLemmatizer().lemmatize(word,'v')

def prepare_text(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 2] #check what works best 
    #tokens = [porter.stem(token) for token in tokens] #stemming
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [lemma1(token) for token in tokens] #lemmatization 
    tokens = [lemma2(token) for token in tokens] #lemmatization verbs
    return tokens

def prepare_topics(topics):
    for topic in topics: 
        prepared_words = [] 
        for string in topics[topic]:
            string = prepare_text(string)
            prepared_words.append(string)
            res = []
            [res.append(x) for x in prepared_words if x not in res] 
            res = [' '.join(map(str, i)) for i in res]
        topics[topic] = res
    return topics 

def prepare_tag(tag): 
    tokens = tokenize(tag)
    tokens = [token for token in tokens if len(token) > 2] 
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [lemma1(token) for token in tokens]  
    tokens = [lemma2(token) for token in tokens] 
    return tokens

def sentiment_analyzer(sentence):
    score = analyser.polarity_scores(sentence)
    negative_score = score['neg']
    positive_score = score['pos']
    neutral_score = score['neu']
    difference = abs(negative_score - positive_score)
    if neutral_score > 0.8:
        return 'neutral'
    elif difference <0.1:
        return 'neutral'
    elif negative_score > positive_score:
        return 'negative'
    else:
        return 'positive'

def dutch_translator(text):
    result = translator.translate(text, dest='en',  src='nl')
    return result.text

def eng_translator(text):
    result = translator.translate(text, dest='nl',  src='en')
    return result.text

In [4]:
prepared_topics = prepare_topics(topics)
df_journal['prepared_tag'] = df_journal.apply(lambda row: prepare_tag(row.tag), axis = 1)

NameError: name 'topics' is not defined

In [5]:
def find_topic_tag(text, tag, prepared_topics):
    sim_scores = [] 
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6

        sim_scores.append(sim_score)
    highest_score = max(sim_scores)
    if highest_score < 3:
        return 'no topic'
    else:
        index_highest_score = list_duplicates_of(sim_scores, highest_score)

        chosen_topics = [] 
        for i in index_highest_score:
            topic =  list(prepared_topics)[i]
            chosen_topics.append(topic.strip()) 
        return chosen_topics

def list_duplicates_of(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

def social_media_score(text, tag, prepared_topics):
    sim_scores = []
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6
        sim_scores.append(sim_score)
    return sim_scores[0]

def body_insecurity_score(text, tag, prepared_topics):
    sim_scores = []
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6
        sim_scores.append(sim_score)
    return sim_scores[1]

def social_anxiety_score(text, tag, prepared_topics):
    sim_scores = []
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6
        sim_scores.append(sim_score)
    return sim_scores[2]

def friendship_score(text, tag, prepared_topics):
    sim_scores = []
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6
        sim_scores.append(sim_score)
    return sim_scores[3]


def find_sentiment(text):
    sentiment = sentiment_analyzer(text)
    return sentiment 

def add_topic_score_to_df(prepared_topics, df_journal):
    df_journal['social_media'] = df_journal.apply(lambda row: social_media_score(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)
    df_journal['body_insecurity'] = df_journal.apply(lambda row: body_insecurity_score(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)
    df_journal['social_anxiety'] = df_journal.apply(lambda row: social_anxiety_score(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)
    df_journal['friendship'] = df_journal.apply(lambda row: friendship_score(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)
    df_journal['topic'] = df_journal.apply(lambda row: find_topic_tag(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)
    df_journal['sentiment'] = df_journal.apply(lambda row: find_sentiment(row.description_eng), axis = 1)

In [6]:
add_topic_score_to_df(prepared_topics, df_journal)
df_journal['check_topics'] = df_journal.apply(lambda row: row.gold_label in row.topic, axis=1)
score = df_journal['check_topics'].value_counts()
correct = score[1]
incorrect = score[0]
percentage_correct = round(((correct/len(df_journal))*100), 2)
percentage_incorrect = round(((incorrect/len(df_journal))*100), 2)
print('The number of correct classifications is:', correct, '. This is', percentage_correct, 'percent of the total amount.', '\nThe number of incorrect classifications is:', incorrect, '. This is', percentage_incorrect, 'percent of the total amount.')
display(df_journal)

NameError: name 'prepared_topics' is not defined

In [7]:
df_journal_topics = df_journal.loc[df_journal['gold_label'] != 'no topic']
score = df_journal_topics['check_topics'].value_counts()
print(score)
correct = score[1]
incorrect = score[0]
percentage_correct = round(((correct/len(df_journal_topics))*100), 2)
percentage_incorrect = round(((incorrect/len(df_journal_topics))*100), 2)
print('The number of correct classifications is:', correct, '. This is', percentage_correct, 'percent of the total amount.', '\nThe number of incorrect classifications is:', incorrect, '. This is', percentage_incorrect, 'percent of the total amount.')
display(df_journal_topics)

NameError: name 'df_journal' is not defined

In [8]:
df_journal_topics1 = df_journal_topics.loc[df_journal['gold_label'] == 'social anxiety']
score = df_journal_topics1['check_topics'].value_counts()
print(score)
correct = score[1]
incorrect = score[0]
percentage_correct = round(((correct/len(df_journal_topics1))*100), 2)
percentage_incorrect = round(((incorrect/len(df_journal_topics1))*100), 2)
print('The number of correct classifications is:', correct, '. This is', percentage_correct, 'percent of the total amount.', '\nThe number of incorrect classifications is:', incorrect, '. This is', percentage_incorrect, 'percent of the total amount.')
display(df_journal_topics1)

NameError: name 'df_journal_topics' is not defined

In [9]:
df_journal_topics = df_journal.loc[df_journal['gold_label'] == 'no topic']
#df_journal_topics = df_journal_topics.loc[df_journal_topics['check_topics'] == 'False']
score = df_journal_topics['check_topics'].value_counts()
print(score)
correct = score[1]
incorrect = score[0]
percentage_correct = round(((correct/len(df_journal_topics))*100), 2)
percentage_incorrect = round(((incorrect/len(df_journal_topics))*100), 2)
print('The number of correct classifications is:', correct, '. This is', percentage_correct, 'percent of the total amount.', '\nThe number of incorrect classifications is:', incorrect, '. This is', percentage_incorrect, 'percent of the total amount.')
display(df_journal_topics)

NameError: name 'df_journal' is not defined

In [10]:
df_journal.groupby(by='gold_label').count()

NameError: name 'df_journal' is not defined

In [11]:
#kijkt naar het percentage van de punten per topic en de kans op een topic is even groot als dit percentage (met 20% willekeurig)
def topics_per_user1(df_journal, df_exercises, user_id):
    list1 = []
    list1.append(user_id)
    df_journal = df_journal[df_journal.id_user.isin(list1)]
    topic_df = df_journal[['social_media', 'social_anxiety', 'body_insecurity', 'friendship']]
    topics = list(topic_df.columns)
    sum_df = topic_df.sum()
    total = sum_df.sum()
    percentages = []
    for i in sum_df:
        percentage = (i / total * 100)
        percentages.append(percentage)
    percentages1 = np.cumsum(percentages)
    dicts = dict(zip(topics, percentages1))    
    sorted_dict = dict(sorted(dicts.items(), key=lambda item: item[1]))
    random1 = randint(0,100)
    if random1 < list(sorted_dict.values())[0]: 
        chosen_topic1 = list(sorted_dict.keys())[0]
    elif random1 < list(sorted_dict.values())[1]: 
        chosen_topic1 = list(sorted_dict.keys())[1]
    elif random1 < list(sorted_dict.values())[2]: 
        chosen_topic1 = list(sorted_dict.keys())[2]
    elif random1 < list(sorted_dict.values())[3]: 
        chosen_topic1 = list(sorted_dict.keys())[3]
    else: 
        chosen_topic1 = random.choice(topics)
    chosen_question_id = return_questions1(df_exercises, user_id, chosen_topic1)
    return chosen_question_id

def pick_random_question(all_ids, likely_ids):
    random_nr = random.uniform(0, 1)
    if random_nr > 0.8:
        chosen_question = random.choice(all_ids)
    else:
        chosen_question = random.choice(likely_ids)
    return chosen_question 

def return_questions1(df_exercises, user_id, chosen_topic1):
    df = df_exercises[df_exercises.Topic == chosen_topic1]
    all_ids = list(df_exercises['ID'])
    likely_ids = list(df['ID'])
    chosen_question_id = pick_random_question(all_ids, likely_ids)
    return(chosen_question_id)

def question_for_user(user_id, df_journal, df_exercises):
    chosen_question_id = topics_per_user1(df_journal, df_exercises, user_id)
    print(chosen_question_id)

question_for_user(11, df_journal, df_exercises)

NameError: name 'df_journal' is not defined

In [10]:
all_dutch_topics = [] 
all_topics = ['social media', 'social anxiety', 'body insecurity', 'friendship']
for topic in all_topics:
    topic = eng_translator(topic)
    all_dutch_topics.append(topic)

df_journal_did = df_journal[df_journal.type == 'did']
df_journal_wish = df_journal[df_journal.type == 'wish']

In [11]:
#did/wish toevoegen en comment 
def comment(user_id, good_topics, all_dutch_topics):
    good_topics_dutch = []
    for topic in good_topics: 
        topic = topic.replace('_', ' ')
        topic = eng_translator(topic)
        good_topics_dutch.append(topic)

    less_topics = [x for x in all_dutch_topics if x not in good_topics_dutch]
 
    if len(less_topics) == 0:
        print('We merken dat je zelfcompassie al erg goed toepast, ga zo door!')
    elif len(good_topics) == 0:
        print('Probeer zelfcompassie toe te passen in je dagelijke leven!')
    elif len(good_topics) == 1:
        less_topic = random.choice(less_topics)
        print('Goed bezig! We merken dat je zelfcompassie al goed toepast of het gebied van', good_topics_dutch[0], ', probeer zelfcompassie ook eens toe te passen op het gebied van', less_topic, '!' )
    elif len(good_topics) == 2:
        less_topic = random.choice(less_topics)
        print('Goed bezig! We merken dat je zelfcompassie al goed toepast of het gebied van', good_topics_dutch[0], 'en', good_topics_dutch[1], '! Probeer zelfcompassie ook eens toe te passen op het gebied van', less_topic, '.' )
    elif len(good_topics) == 3: 
        less_topic = random.choice(less_topics)
        print('Goed bezig! We merken dat je zelfcompassie al goed toepast of het gebied van', good_topics_dutch[0], ', ', good_topics_dutch[1], 'en', good_topics_dutch[2], '! Probeer zelfcompassie ook eens toe te passen op het gebied van', less_topic, '.' )

def comment_for_user(user_id, df_journal_did, all_topics_dutch): 
    df_user = df_journal_did[df_journal_did.id_user == user_id]
    topic_df = df_user[['social_media', 'social_anxiety', 'body_insecurity', 'friendship']]
    sum_df = topic_df.sum()
    good_topics = []
    for i in range(len(sum_df)): 
        if sum_df[i] > 2:
            good_topics.append(topic_df.columns[i])
    comment(user_id, good_topics, all_dutch_topics)
        
comment_for_user(13, df_journal_did, all_dutch_topics)

KeyError: "None of [Index(['social_media', 'social_anxiety', 'body_insecurity', 'friendship'], dtype='object')] are in the [columns]"

In [12]:
display(df_journal_did)

NameError: name 'df_journal_did' is not defined

In [68]:
translator = Translator()
results =translator.translate('हॅलो वर्ल्ड')
print(results.text)

Hello World
